In [6]:
import pandas as pd
import numpy as np
import csv
from DeepSvDDClass import *
import torch

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cuda


In [15]:
#don't use validation dataset since there is no overfitting in svdd?
def train_test(deepsvdd_datapath = None, split_ratio:float = 0.2):
    deepsvdd_data_csv = pd.read_csv(deepsvdd_datapath)
    
    #train data only needs normal data, test data need normal and abnormal data
    data_normal = deepsvdd_data_csv[(deepsvdd_data_csv["normal/abnormal"] == "normal" )]
    data_abnormal = deepsvdd_data_csv[(deepsvdd_data_csv["normal/abnormal"] == "abnormal" )]

    train_dataset_normal, test_dataset_normal  = train_test_split(data_normal, test_size=split_ratio, shuffle = False)
    
    train_dataset = train_dataset_normal
    test_dataset = pd.concat([test_dataset_normal, data_abnormal])
    
    return shuffle(train_dataset), shuffle(test_dataset)


datapath = r'C:\Users\brech\THESIS_local\ToyADMOS\DeepSVDD_case1_ch1.csv'
train_dataset, test_dataset = train_test(deepsvdd_datapath = datapath, split_ratio=0.2)